In [86]:

import pandas as pd 
import transformers




def read_custom_csv(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        file_content = file.read()

        #
        file_content = file_content.replace('\r\n', '\n')

        lines = file_content.split('\n')
        data = [line.split('\t') for line in lines if line.strip() != '']

        # Ensure each row has exactly 2 columns
        data = [row if len(row) == 2 else [row[0], ''] for row in data]

    return pd.DataFrame(data, columns=['abbreviation', 'context'])


train_data_path = "C:/Users/kaczm/OneDrive/Pulpit/Abbr_env_v2/in_train_a.tsv"
df1 = read_custom_csv(train_data_path)


train_data_path = 'C:/Users/kaczm/OneDrive/Pulpit/Abbr_env_v2/expected_train_a.tsv'
df2 = pd.read_csv(train_data_path, sep='\t', header=None, names=['expanded_abbreviation', 'base_abbreviation'])


training_df = pd.merge(df1, df2, left_index=True, right_index=True)


In [87]:
import morfeusz2

morf = morfeusz2.Morfeusz()

# Funkcja do znajdowania potencjalnych rozszerzeń skrótów za pomocą Morfeusza
def find_potential_expansions(abbreviation, morfeusz_analyzer):
    results = morfeusz_analyzer.analyse(abbreviation)
    expansions = set()
    for result in results:
        expansions.add(result[2][1])  # Dodawanie podstawowej formy słowa
    return list(expansions)

# Stosowanie funkcji do kolumny 'abbreviation' w DataFrame
training_df['potential_expansions'] = training_df['abbreviation'].apply(lambda x: find_potential_expansions(x, morf))


In [88]:
training_df[['abbreviation', 'expanded_abbreviation','potential_expansions']]

,abbreviation,expanded_abbreviation,potential_expansions
0,l.,litra,"[litr, lata, ., L]"
1,s.,sekundach,"[., strona, S, sekunda, syn, siostra]"
2,Sen.,Sen.,"[., sen:Sm3, Sen:Sm1, senior, sen:Sm2.m3, Sen:Sf]"
3,s.,siostra,"[., strona, S, sekunda, syn, siostra]"
4,b.,byłego,"[bit, ., brat, bieżący, B, były, bez, bardzo]"
...,...,...,...
4293,s.,strona,"[., strona, S, sekunda, syn, siostra]"
4294,l.,liczba,"[litr, lata, ., L]"
4295,bo.,bo.,"[bo:T, bo:M, .]"
4296,a.,a.,"[., a:P, A, a:I, a:C, a:T, albo]"


In [89]:
# Spróbujmy też znaleźć konteksty wystepujące wokół naszych skrótów
# Zastąpienie <mask> skrótem
training_df['context_with_abbreviation'] = training_df.apply(lambda x: x['context'].replace('<mask>', x['abbreviation']), axis=1)


In [90]:
pd.set_option('display.max_colwidth', None)
training_df

,abbreviation,context,expanded_abbreviation,base_abbreviation,potential_expansions,context_with_abbreviation
0,l.,"ciągników serii 1523 i 1221 otrzymały sześciocylindrowe silniki o pojemności 7,2 <mask> Także te modele nie były wysilone mocowo i osiągały 158 KM w przypadku Belarusa",litra,litr,"[litr, lata, ., L]","ciągników serii 1523 i 1221 otrzymały sześciocylindrowe silniki o pojemności 7,2 l. Także te modele nie były wysilone mocowo i osiągały 158 KM w przypadku Belarusa"
1,s.,"na zderzaku poprzedzającego auta w odległości zaledwie 1,235 s przy założonych 3 <mask> Przy okazji prawą ręką obsługiwał telefon. Podobnie srebrny Ford S-Max – jego szofer",sekundach,sekunda,"[., strona, S, sekunda, syn, siostra]","na zderzaku poprzedzającego auta w odległości zaledwie 1,235 s przy założonych 3 s. Przy okazji prawą ręką obsługiwał telefon. Podobnie srebrny Ford S-Max – jego szofer"
2,Sen.,"""Kambodża nie wypowiada się na temat wewnętrznych spraw innych państw"" - powiedział Hun <mask> Agencja Reutera przypomina, że rządzący Kambodżą od ponad 30 lat polityk sam w 1997",Sen.,Sen.,"[., sen:Sm3, Sen:Sm1, senior, sen:Sm2.m3, Sen:Sf]","""Kambodża nie wypowiada się na temat wewnętrznych spraw innych państw"" - powiedział Hun Sen. Agencja Reutera przypomina, że rządzący Kambodżą od ponad 30 lat polityk sam w 1997"
3,s.,"grzechów. Cykl rozpoczęliśmy rozmową z Jerzym Pilchem o pysze; o gniewie mówiła <mask> Małgorzata Chmielewska, o chciwości - Janusz Palikot, o nieumiarkowaniu - Bartłomiej",siostra,siostra,"[., strona, S, sekunda, syn, siostra]","grzechów. Cykl rozpoczęliśmy rozmową z Jerzym Pilchem o pysze; o gniewie mówiła s. Małgorzata Chmielewska, o chciwości - Janusz Palikot, o nieumiarkowaniu - Bartłomiej"
4,b.,"ocenił w rozmowie z PAP poseł Michał Jach. Odniósł się w ten sposób do wypowiedzi <mask> szefa MON Tomasza Siemoniaka, który podczas środowej wizyty w Rzeszowie powiedział:",byłego,były,"[bit, ., brat, bieżący, B, były, bez, bardzo]","ocenił w rozmowie z PAP poseł Michał Jach. Odniósł się w ten sposób do wypowiedzi b. szefa MON Tomasza Siemoniaka, który podczas środowej wizyty w Rzeszowie powiedział:"
...,...,...,...,...,...,...
4293,s.,"bywa męczący. Poniżej dalsza część artykułu [i]Prószyński i S-ka, Warszawa 2010 <mask> 383[/i] © Licencja na publikację © ℗ Wszystkie prawa zastrzeżone Źródło: Rzeczpospolita",strona,strona,"[., strona, S, sekunda, syn, siostra]","bywa męczący. Poniżej dalsza część artykułu [i]Prószyński i S-ka, Warszawa 2010 s. 383[/i] © Licencja na publikację © ℗ Wszystkie prawa zastrzeżone Źródło: Rzeczpospolita"
4294,l.,"Lewicy Demokratycznej - 12,5 proc. (l. głosów - 5 950), Konfederacja - 7,76 proc (<mask> głosów - 3 694), Polskie Stronnictwo Ludowe - 4,49 proc. (l. głosów - 2 137). W",liczba,liczba,"[litr, lata, ., L]","Lewicy Demokratycznej - 12,5 proc. (l. głosów - 5 950), Konfederacja - 7,76 proc (l. głosów - 3 694), Polskie Stronnictwo Ludowe - 4,49 proc. (l. głosów - 2 137). W"
4295,bo.,www. wloclawek.pl na stronie dedykowanej włocławskiemu budżetowi obywatelskiemu: <mask> wloclawek.eu oraz na tablicy ogłoszeń w urzędzie miasta. Głosowanie przeprowadzone,bo.,bo.,"[bo:T, bo:M, .]",www. wloclawek.pl na stronie dedykowanej włocławskiemu budżetowi obywatelskiemu: bo. wloclawek.eu oraz na tablicy ogłoszeń w urzędzie miasta. Głosowanie przeprowadzone
4296,a.,"przewodnictwem Prezydenta RP a we Francji nie Zadanie 16. 16.1 funkcja kontrolna 16.2 <mask> politycznej b. premiera c. 231 Zadanie 17. 17. 1 A,E 17.2. C,D Zadanie 18 a. Rada",a.,a.,"[., a:P, A, a:I, a:C, a:T, albo]","przewodnictwem Prezydenta RP a we Francji nie Zadanie 16. 16.1 funkcja kontrolna 16.2 a. politycznej b. premiera c. 231 Zadanie 17. 17. 1 A,E 17.2. C,D Zadanie 18 a. Rada"


In [91]:
# Przetwarzanie tekstu - TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans

# Ścieżka do pliku z polskimi słowami stopu
stop_words_file_path = "C:/Users/kaczm/OneDrive/Pulpit/Abbr_env_v2/polish.stopwords.txt"

# Wczytanie listy słów stopu z pliku
with open(stop_words_file_path, 'r', encoding='utf-8') as file:
    polish_stop_words = [line.strip() for line in file.readlines()]


tfidf_vectorizer = TfidfVectorizer(stop_words=polish_stop_words)
X_tfidf = tfidf_vectorizer.fit_transform(training_df['context_with_abbreviation'])





In [92]:
# Klasteryzacja 
num_clusters = 50  
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
kmeans.fit(X_tfidf)

# Przypisanie klastrów do danych
training_df['cluster'] = kmeans.labels_

C:\Users\kaczm\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [93]:
# A teraz analiza morfologiczna

def tag_morfeusz_simple(text, analyzer):
    analysis = analyzer.analyse(text)
    tagged = [(word[2][1], word[2][2].split(':')[0]) for word in analysis]  # Zwracamy część mowy i formę podstawową
    return tagged

context_to_tag = training_df['context_with_abbreviation']  

# Tagowanie
training_df['tagged_context'] = context_to_tag.apply(lambda x: tag_morfeusz_simple(x, morf))


In [96]:
training_df.dtypes

abbreviation                 object
context                      object
expanded_abbreviation        object
base_abbreviation            object
potential_expansions         object
context_with_abbreviation    object
cluster                       int32
tagged_context               object
dtype: object